In [1]:
import pandas as pd

In [2]:
df  = pd.read_excel("ReviewsFileName.xlsx")

In [3]:
df.head()

,Review,Sentiment
0,the rock is destined to be the 21st century's ...,0
1,"the gorgeously elaborate continuation of "" the...",0
2,effective but too-tepid biopic\n,0
3,if you sometimes like to go to the movies to h...,0
4,"emerges as something rare , an issue movie tha...",0


In [4]:
len(df)

10662

In [5]:
df.head()

,Review,Sentiment
0,the rock is destined to be the 21st century's ...,0
1,"the gorgeously elaborate continuation of "" the...",0
2,effective but too-tepid biopic\n,0
3,if you sometimes like to go to the movies to h...,0
4,"emerges as something rare , an issue movie tha...",0


In [6]:
df.Review[0]

'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . \n'

In [7]:
from nltk.stem import PorterStemmer
import re
from nltk.corpus import stopwords
import numpy as np
cachedStopWords = stopwords.words("english")

stemmer = PorterStemmer()


def pre_process_data(data, remove_stop_words=True):

    list_Sentiment = []
    list_Review = []
    len_data = len(df)
    i=0
    
    for row in df.iterrows():
        i+=1
        if i % 500 == 0:
            print("%s | %s rows" % (i, len_data))

        ##### Remove and clean comments
        posts = row[1].Review
        temp = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', posts)
        temp = re.sub("[^a-zA-Z]", " ", temp)
        temp = re.sub(' +', ' ', temp).lower()
        if remove_stop_words:
            temp = " ".join([stemmer.stem(w) for w in temp.split(' ') if w not in cachedStopWords])
        else:
            temp = " ".join([stemmer.stem(w) for w in temp.split(' ')])
        list_Sentiment.append(row[1].Sentiment)
        list_Review.append(temp)

    #del data
    list_Review = np.array(list_Review)
    list_Sentiment = np.array(list_Sentiment)
    return list_Review, list_Sentiment

In [8]:
list_Review, list_Sentiment = pre_process_data(df, remove_stop_words=True)

500 | 10662 rows
1000 | 10662 rows
1500 | 10662 rows
2000 | 10662 rows
2500 | 10662 rows
3000 | 10662 rows
3500 | 10662 rows
4000 | 10662 rows
4500 | 10662 rows
5000 | 10662 rows
5500 | 10662 rows
6000 | 10662 rows
6500 | 10662 rows
7000 | 10662 rows
7500 | 10662 rows
8000 | 10662 rows
8500 | 10662 rows
9000 | 10662 rows
9500 | 10662 rows
10000 | 10662 rows
10500 | 10662 rows


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
cntizer = CountVectorizer()
X_cnt = cntizer.fit_transform(list_Review)

In [10]:
reverse_dic = {}
for key in cntizer.vocabulary_:
    reverse_dic[cntizer.vocabulary_[key]] = key

In [11]:
top_30 = np.asarray(np.argsort(np.sum(X_cnt, axis=0))[0,-30:][0, ::-1]).flatten()
[reverse_dic[v] for v in top_30]

['film',
 'movi',
 'like',
 'one',
 'make',
 'stori',
 'time',
 'charact',
 'good',
 'comedi',
 'even',
 'much',
 'work',
 'perform',
 'feel',
 'well',
 'way',
 'director',
 'get',
 'look',
 'littl',
 'love',
 'funni',
 'come',
 'enough',
 'never',
 'life',
 'take',
 'bad',
 'may']

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(list_Review, list_Sentiment, test_size=0.33, random_state=42)

In [13]:
X_train.shape

(7143,)

In [14]:
X_test.shape

(3519,)

In [37]:
X_train

array(['violent initi rite audienc much angeliqu open danc guarante karmen enthron among cinema memor women ',
       'fill low brow humor gratuit violenc disturb disregard life ',
       'transport live fun unapologet dumb ', ...,
       'watch chemistri freeman judd howev almost make movi worth see almost ',
       'eleg exquisit modul psycholog thriller ',
       'charact zigzag suffici develop support film construct around '],
      dtype='<U210')

In [38]:
import sklearn
sklearn.__version__

'0.22'

In [17]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfizer = TfidfVectorizer(ngram_range= (1,2),max_df=0.3,min_df=7)
# X_tfidf =  tfizer.fit_transform(X_train).toarray()

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

In [22]:
clf=Pipeline([
        ('tfidf_vectorizer', TfidfVectorizer(ngram_range= (1,2),max_df=0.3,min_df=7)),
        ('rf_classifier', RandomForestClassifier(n_estimators=500,verbose=1,n_jobs=-1))
    ])

In [25]:
clf.fit(X_train,y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    3.6s finished


Pipeline(memory=None,
         steps=[('tfidf_vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.3, max_features=None,
                                 min_df=7, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 toke...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None

In [26]:
y_pred=clf.predict(X_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished


In [27]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[1253,  525],
       [ 498, 1243]], dtype=int64)